### Import packages

Note that some packages are local python scripts created to enable the proper use of the Notebook

In [3]:
import numpy as np
import torch

from torch_geometric.loader import LinkNeighborLoader

import matplotlib.pyplot as plt


### Basic configurations

Define the main options for the Training and execution

In [ ]:
import glob
from os import path as osp
import json

#CONFIG EXPERIMENT
SPLIT = "80/10/10"
BATCH = "_batch"
ADD_NEGATIVE_TRAINING = False
F1_TR = 0.5

# important directories
DATASET_LABEL = "AB_new"

data_path = osp.join('../data/', DATASET_LABEL)
trained_path = osp.join('../trained/', DATASET_LABEL)
save_path_csv = osp.join('../results/', DATASET_LABEL)

save_path_csv = osp.join(save_path_csv, 'results' + BATCH + '.csv')

raw_path = osp.join(data_path, 'raw')

# Read JSON to get extra parameters in the graph
json_path = glob.glob(osp.join(raw_path, "Parameters.json"))
parameters_exist = len(json_path) != 0
parameters_for_gnn = None
if parameters_exist:
    parameters_for_gnn = json_path[0]

with open(parameters_for_gnn) as json_data:
    parameters = json.load(json_data)

class_left = parameters['CLASS_LEFT']
class_right = parameters['CLASS_RIGHT']
relation_name = parameters['RELATION_NAME']
rev_relation_name = "rev_" + relation_name
EPOCHS = parameters['EPOCHS']
LEARNING_RATE = parameters['LEARNING_RATE']

UNIQUE_ID_LEFT = parameters['UNIQUE_ID_LEFT']
UNIQUE_ID_RIGHT = parameters['UNIQUE_ID_RIGHT']

IGNORE_ATTRIBUTES_LEFT = parameters['IGNORE_ATTRIBUTES_LEFT']
IGNORE_ATTRIBUTES_RIGHT = parameters['IGNORE_ATTRIBUTES_RIGHT']

STRINGS_ENCODER = parameters['STRINGS_ENCODER']


### Print functions for the performance curves

Functions to print both ROC (Receiver Operating Characteristic) curve and PR (Precision-Recall) curve

In [ ]:
def print_pr_curve(precision, recall, title, pr_auc, no_skill):
    #create precision recall curve
    _, ax = plt.subplots()
    ax.plot(recall, precision, color='purple', label='AUC = %0.4f' % pr_auc)
    ax.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')

    #add axis labels to plot
    ax.set_title(title)
    ax.set_ylabel('Precision')
    ax.set_xlabel('Recall')

    plt.legend(loc='best')
    #display plot
    plt.show

def print_roc_curve(fpr, tpr, title, roc_auc):
    #create ROC curve
    _, ax = plt.subplots()
    ax.plot(fpr, tpr, color='purple', label='AUC = %0.4f' % roc_auc)
    ax.plot([0, 1], [0, 1], linestyle='--', label='No Skill')

    #add axis labels to plot
    ax.set_title(title)
    ax.set_ylabel('True Positive Rate(TPR)')
    ax.set_xlabel('False Positive Rate(FPR)')

    plt.legend(loc='best')
    #display plot
    plt.show()

### Define directories to get the data and to save the results of the proccess

In [ ]:

folders = {
    "rule_1" : "A.a = B.b",
    # "rule_2" : "A.a = B.b + B.c",
}

### Configuration of EMF models

In [ ]:
from pyecore.resources import ResourceSet, URI

# Custom function get Heterodata from EMF model
from utils.ab import Func

# Register the metamodels in the resource set    
resource_set = ResourceSet()
ecore_path_a = glob.glob(osp.join(data_path, 'metamodels/A.ecore'))[0] # only one metamodel
ecore_path_b = glob.glob(osp.join(data_path, 'metamodels/B.ecore'))[0] # only one metamodel   
resource_a = resource_set.get_resource(URI(ecore_path_a))
mm_root_a = resource_a.contents[0]
resource_b = resource_set.get_resource(URI(ecore_path_b))
mm_root_b = resource_b.contents[0]

resource_set.metamodel_registry[mm_root_a.nsURI] = mm_root_a
resource_set.metamodel_registry[mm_root_b.nsURI] = mm_root_b

# TODO: Consider the ignore attributes from JSON
dataset_func = Func(sentence_encoding_name=STRINGS_ENCODER, features_to_encode_left=['a', 's'], features_to_encode_right=['b', 'c', 'd','s'], unique_id_left=UNIQUE_ID_LEFT, unique_id_right=UNIQUE_ID_RIGHT)

### Create the Hetero data

In [ ]:
directory_rule_name = "rule_1"
# Register the models in the resource set    
xmi_path_left = glob.glob(osp.join(raw_path, directory_rule_name, "DatasetLeft.xmi"))[0]
m_resource_left = resource_set.get_resource(URI(xmi_path_left))
model_root_left = m_resource_left.contents

xmi_path_right = glob.glob(osp.join(raw_path, directory_rule_name, "DatasetRight.xmi"))[0]
m_resource_right = resource_set.get_resource(URI(xmi_path_right))
model_root_right = m_resource_right.contents

relations_path = glob.glob(osp.join(raw_path, directory_rule_name, "Relations.csv"))
relations_exist = len(relations_path) != 0
relations_for_graph = None
if relations_exist:
    relations_for_graph = relations_path[0]   

data, left_original_mapping, right_original_mapping = dataset_func.xmi_to_graph(model_root_left, model_root_right, relations_for_graph, parameters_for_gnn)

print(data)

### Change device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

### Add reverse relation

In [ ]:
import torch_geometric.transforms as T

 # Add a reverse (class_right, rev_relation_name, class_left) relation for message passing:
data = T.ToUndirected()(data)
del data[class_right, rev_relation_name, class_left].edge_label  # Remove "reverse" label.

### Split the data

In [ ]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1, # 10% of links for validation
    num_test=0.1, # 10% os links for test
    disjoint_train_ratio=0.3, # use 70% for message passing and 30% for supervision
    neg_sampling_ratio=1.0, # generate 2:1 negative edges
    add_negative_train_samples=ADD_NEGATIVE_TRAINING,
    edge_types=(class_left, relation_name, class_right),
    rev_edge_types=(class_right, rev_relation_name, class_left)
)(data)

edge_label_index = train_data[class_left, relation_name, class_right].edge_label_index
edge_label = train_data[class_left, relation_name, class_right].edge_label

print(train_data)
print(edge_label_index)
print(edge_label)

### Train loader

In [ ]:
# LOADERS
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=1.0,
    edge_label_index=((class_left, relation_name, class_right), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

### GNN architecture

Two SageConv Layers for Node embedding and a Classifier that performs the dot-product between embeddings. It also use a extra `to_hetero` transformer since it will be applied to the HeteroData objects

In [ ]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    """
    This class is responsible for creating the ML model
    """

    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        """
        This method is responsible for executing the forward pass of the convolutional GNN.

        Parameters
        ----------
        x: torch.Tensor
            input tensor
        edge_index: torch.Tensor
            edge index

        Returns
        -------
        x: torch.Tensor
            output tensor
        """
        # A 2-layer GNN computation graph.
        # `ReLU` is the non-lineary function used in-between.
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    """
    The final classifier applies the dot-product between source and destination node embeddings to derive edge-level predictions
    """

    def forward(self, x_left: torch.Tensor, x_right: torch.Tensor, edge_label_index: torch.Tensor) -> torch.Tensor:
        """
        This method is responsible for executing the forward pass of the classifier.
        
        Parameters
        ----------
        x_left: torch.Tensor
            input tensor for Left class
        x_right: torch.Tensor
            input tensor for Right class
        edge_label_index: torch.Tensor
            edge label index

        Returns
        -------
        x: torch.Tensor
            output tensor
        """

        # Convert node embeddings to edge-level representations:
        edge_feat_left  = x_left[edge_label_index[0]]
        edge_feat_right = x_right[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_left * edge_feat_right).sum(dim=-1)


class Model(torch.nn.Module):
    """
    This class is responsible for creating the model with two SageConv layers with a addiotinal classifier to be used in the training.
    """

    def __init__(self, hidden_channels, data):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for A and B:
        self.left_lin = torch.nn.Linear(data[class_left].num_features, hidden_channels)
        self.right_lin = torch.nn.Linear(data[class_right].num_features, hidden_channels)
        # self.left_emb = torch.nn.Embedding(data[class_left].num_nodes, hidden_channels)
        # self.right_emb = torch.nn.Embedding(data[class_right].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)
        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())
        self.classifier = Classifier()

    def forward(self, data) -> torch.Tensor:
        """	
        This method is responsible for executing the forward pass of the model.

        Parameters
        ----------
        data: HeteroData
            input data

        Returns
        -------
        pred: torch.Tensor
            output tensor
        """
        
        x_dict = {
        #   class_left: self.A_lin(data[class_left].x) + self.left_emb(data[class_left].node_id),
          class_left: self.left_lin(data[class_left].x) ,#+ self.left_emb(data[class_left].node_id),
          class_right: self.right_lin(data[class_right].x) ,#+ self.right_emb(data[class_right].node_id),
        } 
        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict[class_left],
            x_dict[class_right],
            data[class_left, relation_name, class_right].edge_label_index,
        )
        return pred
    
    def get_name(self):
        return "dot_product"
    
model = Model(hidden_channels=64, data=data)

print(model)

### Training a Heterogeneous Link-level GNN

Training the GNN is similar to training any PyTorch model.
We move the model to the desired device, and initialize an optimizer that takes care of adjusting model parameters via stochastic gradient descent.

The training loop then iterates over the training mini-batches, applies the forward computation of the model, computes the loss from ground-truth labels and obtained predictions using binary cross entropy, and adjusts model parameters via back-propagation.

In [ ]:
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=float(LEARNING_RATE))

for epoch in range(1, int(EPOCHS)):
    total_loss = total_examples = 0
    for _, sampled_data in enumerate(train_loader):
        optimizer.zero_grad()

        sampled_data.to(device)
        pred = model(sampled_data)

        ground_truth = sampled_data[class_left, relation_name, class_right].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    if epoch % 50 == 0:
        print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

### Evaluating a Heterogeneous Link-level GNN

After training, we evaluate our model on useen data coming from the validation set.

In [ ]:
# Define the validation seed edges:
edge_label_index = val_data[class_left, relation_name, class_right].edge_label_index
edge_label = val_data[class_left, relation_name, class_right].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(
        (class_left, relation_name, class_right), 
        val_data[(class_left, relation_name, class_right)].edge_label_index,
    ),
    edge_label=val_data[(class_left, relation_name, class_right)].edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data[class_left, relation_name, class_right].edge_label_index.size(1) == 3 * 128
assert sampled_data[class_left, relation_name, class_right].edge_label.min() >= 0
assert sampled_data[class_left, relation_name, class_right].edge_label.max() <= 1

In [ ]:
import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc as area_curve
from sklearn.metrics import f1_score

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))
        ground_truths.append(sampled_data[class_left, relation_name, class_right].edge_label)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

print("Sample of y_true and y_score")
print(ground_truth[:10])
print(pred[:10])

auc = roc_auc_score(ground_truth, pred)
print(f"Validation AUC: {auc:.4f}")

fpr, tpr, roc_tr = roc_curve(ground_truth, pred)
print_roc_curve(fpr, tpr, DATASET_LABEL + "_Validation", auc)

# no_skill = len(ground_truth[ground_truth==1]) / len(ground_truth)
# precision, recall, _ = precision_recall_curve(ground_truth, (torch.sigmoid(pred)))
# pr_auc = area_curve (recall, precision)
#print_pr_curve(precision, recall, DATASET_LABEL + "_Validation", pr_auc)

#f1 = f1_score(ground_truth, pred_labels)

In [ ]:
import pandas as pd


roc_auc = area_curve(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

####################################
# The optimal cut off would be where tpr is high and fpr is low
# tpr - (1-fpr) is zero or near to zero is the optimal cut off point
####################################
i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({
    'fpr' : pd.Series(fpr, index=i),
    'tpr' : pd.Series(tpr, index = i), 
    '1-fpr' : pd.Series(1-fpr, index = i), 
    'tf' : pd.Series(tpr - (1-fpr), index = i), 
    'thresholds' : pd.Series(roc_tr, index = i)})

# Plot tpr vs 1-fpr
fig, ax = plt.subplots()
plt.plot(roc['tpr'])
plt.plot(roc['1-fpr'], color = 'red')
plt.xlabel('1-False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
ax.set_xticklabels([])
plt.show()

print(roc.iloc[(roc.tf-0).abs().argsort()[:1]]['thresholds'])

### Predict results to be used in the new view

In [ ]:
from tqdm.auto import tqdm

model = model.cpu() 
model.eval() 

total_class_left = len(left_original_mapping) 
total_class_right = len(right_original_mapping)

threshold = 0.7
predictions = {}

for uri_idenifier_left, left_id in tqdm(left_original_mapping.items()):
    predictions[uri_idenifier_left] = []

    left_row = torch.tensor([left_id] * total_class_right) 
    all_right_ids = torch.arange(total_class_right) 
    edge_label_index = torch.stack([left_row, all_right_ids], dim=0) 
    data[class_left, relation_name, class_right].edge_label_index = edge_label_index
    
    with torch.no_grad(): 
        pred = model(data) 
    probabilities = torch.sigmoid(pred)
    pred_labels = (probabilities > threshold).long()

    recommended_links = all_right_ids[pred_labels == 1].tolist()

    predictions[uri_idenifier_left] = recommended_links
    #print("The {} with ID {} should be linked with the following {}:\n{}".format(class_left, uri_idenifier_left, class_right, predictions[uri_idenifier_left]), end="\n\n")


### Save results in JSON to enable the definition of WeavingModel

In [ ]:
predicted_path = osp.join('../predicted/', DATASET_LABEL, model.get_name() + BATCH + "_" + directory_rule_name + ".json")

inv_right_mapping = {v: k for k, v in right_original_mapping.items()}
json_dict = {}
#iterate over the predictions and create the JSON
for uri_idenifier_left, potential_links in predictions.items():
    json_dict[uri_idenifier_left] = [inv_right_mapping[x] for x in potential_links]

with open(predicted_path, 'w+') as f:
    json.dump(json_dict, f)


### Save metrics and Hyperparameters in CSV

In [ ]:
new_file = not osp.exists(save_path_csv)

if new_file:
    open(save_path_csv, 'w+').write(f"'Best', 'Rule', 'Desc', 'ML_Arch_Name', 'encode_features_A', 'encode_features_B', 'Training_split', 'Epochs', 'Training_negative_edges', 'Learning Rate', 'Final validation score (ROC_AUC)'\n")


with open(save_path_csv, 'a') as csvfile:

    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # result line
    filewriter.writerow(["", directory_rule_name, folders[directory_rule_name], model.get_name(), ",".join(['a']), ",".join(["b"]), SPLIT, EPOCHS, str(ADD_NEGATIVE_TRAINING), LEARNING_RATE, '{0:.{1}f}'.format(val_roc_auc, 4)])


trained_path = osp.join(trained_path, model.get_name() + BATCH + "_" + directory_rule_name + ".pth")
torch.save(model.state_dict(), trained_path)
trained_path = osp.join('../trained/', DATASET_LABEL) # reset the name of the file